### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [1]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [2]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

In [3]:
keywords = ["CacheNodeidentityInfo", "EnableSeparateConfigBase", "ExperimentalClusterDNS",
            "GoogleCloudBucketACL", "KeepLaunchConfigurations", "SpecOverrideFlag", "Spotinst",
            "SpotinstOcean", "SpotinstHybrid", "SpotinstController", "VFSVaultSupport",
            "VPCSkipEnableDNSSupport", "SkipEtcdVersionCheck", "TerraformJSON", "ClusterAddons",
            "KopsControllerStateStore", "APIServerNodes", "UseAddonOperators", "AWSIPv6",
            "TerraformManagedFiles", "AlphaAllowGCE", "AlphaAllowALI"#, "Azure" 
            # removing "Azure" because it affects more than 100 files
           ] 

### List files with keywords

#### List all .go files

In [4]:
go_folders = [x[0] for x in os.walk("./kops/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./kops/vendor/github.com/docker/docker/api/types/container/hostconfig_unix.go
./kops/vendor/github.com/docker/distribution/registry/api/v2/urls.go
./kops/vendor/k8s.io/api/apps/v1/generated.pb.go
./kops/vendor/golang.org/x/tools/internal/gocommand/vendor.go
./kops/vendor/github.com/aliyun/alibaba-cloud-sdk-go/services/slb/struct_master_slave_backend_servers_in_create_master_slave_server_group.go
./kops/vendor/github.com/cespare/xxhash/v2/xxhash_safe.go
./kops/vendor/golang.org/x/net/context/context.go
./kops/vendor/k8s.io/client-go/listers/core/v1/node.go
./kops/vendor/k8s.io/apimachinery/pkg/apis/meta/v1/types.go
./kops/vendor/k8s.io/client-go/kubernetes/typed/core/v1/event.go


In [5]:
len(go_files)

8304

#### List the interesting files, containing at least one keyword with the featureflag 

In [6]:
go_file_interests = []

count_file = dict()
occurences = dict()

for kw in sorted(keywords):
    count_file[kw] = 0
    occurences[kw] = 0

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    for kw in keywords:
        if kw in s:
            count_file[kw]+=1
            occurences[kw]+= s.count(kw)
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0 and "featureflag" in s:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./kops/upup/pkg/fi/cloudup/template_functions.go contains :
-> CacheNodeidentityInfo
-> Spotinst
-> APIServerNodes


./kops/upup/pkg/fi/cloudup/apply_cluster.go contains :
-> Spotinst
-> SpotinstHybrid
-> KopsControllerStateStore
-> TerraformManagedFiles
-> AlphaAllowGCE
-> AlphaAllowALI


./kops/upup/pkg/fi/cloudup/populate_instancegroup_spec.go contains :
-> APIServerNodes


./kops/upup/pkg/fi/cloudup/new_cluster.go contains :
-> Spotinst


./kops/upup/pkg/fi/cloudup/alitasks/launchconfiguration.go contains :
-> KeepLaunchConfigurations


./kops/upup/pkg/fi/cloudup/terraform/target_hcl2.go contains :
-> Spotinst


./kops/upup/pkg/fi/cloudup/terraform/target.go contains :
-> TerraformJSON
-> TerraformManagedFiles


./kops/upup/pkg/fi/cloudup/bootstrapchannelbuilder/bootstrapchannelbuilder.go contains :
-> Spotinst
-> SpotinstController
-> ClusterAddons
-> UseAddonOperators


./kops/upup/pkg/fi/cloudup/awsup/aws_cloud.go contains :
-> Spotinst
-> SpotinstHybrid


./kops/upup/pkg/fi/clo

In [7]:
len(go_file_interests)

29

In [8]:
count_file

{'APIServerNodes': 8,
 'AWSIPv6': 3,
 'AlphaAllowALI': 3,
 'AlphaAllowGCE': 7,
 'CacheNodeidentityInfo': 5,
 'ClusterAddons': 3,
 'EnableSeparateConfigBase': 2,
 'ExperimentalClusterDNS': 2,
 'GoogleCloudBucketACL': 2,
 'KeepLaunchConfigurations': 2,
 'KopsControllerStateStore': 2,
 'SkipEtcdVersionCheck': 2,
 'SpecOverrideFlag': 6,
 'Spotinst': 41,
 'SpotinstController': 2,
 'SpotinstHybrid': 5,
 'SpotinstOcean': 2,
 'TerraformJSON': 3,
 'TerraformManagedFiles': 4,
 'UseAddonOperators': 3,
 'VFSVaultSupport': 2,
 'VPCSkipEnableDNSSupport': 2}

In [9]:
occurences

{'APIServerNodes': 16,
 'AWSIPv6': 8,
 'AlphaAllowALI': 6,
 'AlphaAllowGCE': 10,
 'CacheNodeidentityInfo': 10,
 'ClusterAddons': 10,
 'EnableSeparateConfigBase': 4,
 'ExperimentalClusterDNS': 7,
 'GoogleCloudBucketACL': 3,
 'KeepLaunchConfigurations': 4,
 'KopsControllerStateStore': 4,
 'SkipEtcdVersionCheck': 6,
 'SpecOverrideFlag': 9,
 'Spotinst': 162,
 'SpotinstController': 4,
 'SpotinstHybrid': 12,
 'SpotinstOcean': 5,
 'TerraformJSON': 6,
 'TerraformManagedFiles': 7,
 'UseAddonOperators': 6,
 'VFSVaultSupport': 7,
 'VPCSkipEnableDNSSupport': 5}

## Step 3 - Search Feature Toggles dependencies in the AST

In [10]:
count_loc = dict()
for kw in sorted(keywords):
    count_loc[kw] = 0



for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                count_loc[kw]+=get_name(node).count("\n")+1
                print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./kops/upup/pkg/fi/cloudup/template_functions.go 



if featureflag.Spotinst.Enabled() {
		if creds, err := spotinst.LoadCredentials(); err == nil {
			dest["SpotinstToken"] = func() string { return creds.Token }
			dest["SpotinstAccount"] = func() string { return creds.Account }
			dest["SpotinstTokenBase64"] = func() string { return base64.StdEncoding.EncodeToString([]byte(creds.Token)) }
			dest["SpotinstAccountBase64"] = func() string { return base64.StdEncoding.EncodeToString([]byte(creds.Account)) }
		}
	}


-> featureflag.Spotinst.Enabled()


if creds, err := spotinst.LoadCredentials(); err == nil {
			dest["SpotinstToken"] = func() string { return creds.Token }
			dest["SpotinstAccount"] = func() string { return creds.Account }
			dest["SpotinstTokenBase64"] = func() string { return base64.StdEncoding.EncodeToString([]byte(creds.Token)) }
			dest["SpotinstAccountBase64"] = func() string { return base64.StdEncoding.EncodeToString([]byte(creds.Account)) }
		}





File :             ./kops/upup/pkg/fi/cloudup/terraform/target.go 



if featureflag.TerraformJSON.Enabled() {
		if featureflag.TerraformManagedFiles.Enabled() {
			// Terraform's JSON representation doesn't support provider aliases which are required for managed files
			return errors.New("TerraformJSON cannot be used with TerraformManagedFiles")
		}
		err = t.finishJSON()
	} else {
		err = t.finishHCL2()
	}


-> featureflag.TerraformJSON.Enabled()


if featureflag.TerraformJSON.Enabled() {
		if featureflag.TerraformManagedFiles.Enabled() {
			// Terraform's JSON representation doesn't support provider aliases which are required for managed files
			return errors.New("TerraformJSON cannot be used with TerraformManagedFiles")
		}
		err = t.finishJSON()
	} else {
		err = t.finishHCL2()
	}


-> featureflag.TerraformJSON.Enabled()


if featureflag.TerraformManagedFiles.Enabled() {
			// Terraform's JSON representation doesn't support provider aliases which are required for managed files



File :             ./kops/upup/pkg/fi/cloudup/awstasks/vpc.go 



if shared {
		// Verify the VPC was found and matches our required settings
		if a == nil {
			return fmt.Errorf("VPC with id %q not found", fi.StringValue(e.ID))
		}

		if changes != nil && changes.EnableDNSSupport != nil {
			if featureflag.VPCSkipEnableDNSSupport.Enabled() {
				klog.Warningf("VPC did not have EnableDNSSupport=true, but ignoring because of VPCSkipEnableDNSSupport feature-flag")
			} else {
				// TODO: We could easily just allow kops to fix this...
				return fmt.Errorf("VPC with id %q was set to be shared, but did not have EnableDNSSupport=true.", fi.StringValue(e.ID))
			}
		}
	}




if changes != nil && changes.EnableDNSSupport != nil {
			if featureflag.VPCSkipEnableDNSSupport.Enabled() {
				klog.Warningf("VPC did not have EnableDNSSupport=true, but ignoring because of VPCSkipEnableDNSSupport feature-flag")
			} else {
				// TODO: We could easily just allow kops to fix this...
				return fmt.E



File :             ./kops/pkg/wellknownoperators/operators.go 



if !featureflag.UseAddonOperators.Enabled() {
		return nil, nil, nil
	}


-> !featureflag.UseAddonOperators.Enabled()


File :             ./kops/pkg/apis/kops/validation/legacy.go 



if c.Spec.KubeDNS != nil {
		if c.Spec.KubeDNS.ServerIP != "" {
			address := c.Spec.KubeDNS.ServerIP
			ip := net.ParseIP(address)
			if ip == nil {
				allErrs = append(allErrs, field.Invalid(fieldSpec.Child("kubeDNS", "serverIP"), address, "Cluster had an invalid kubeDNS.serverIP"))
			} else {
				if serviceClusterIPRange != nil && !serviceClusterIPRange.Contains(ip) {
					allErrs = append(allErrs, field.Forbidden(fieldSpec.Child("kubeDNS", "serverIP"), fmt.Sprintf("ServiceClusterIPRange %q must contain the DNS Server IP %q", c.Spec.ServiceClusterIPRange, address)))
				}
				if !featureflag.ExperimentalClusterDNS.Enabled() {
					if isExperimentalClusterDNS(c.Spec.Kubelet, c.Spec.KubeDNS) {
						allErrs = append(allErrs, field.For



File :             ./kops/cmd/kops/edit_cluster.go 



if featureflag.SpecOverrideFlag.Enabled() {
		cmd.Flags().StringSliceVar(&options.Sets, "set", options.Sets, "Directly set values in the spec")
		cmd.RegisterFlagCompletionFunc("set", func(cmd *cobra.Command, args []string, toComplete string) ([]string, cobra.ShellCompDirective) {
			return nil, cobra.ShellCompDirectiveNoFileComp
		})
		cmd.Flags().StringSliceVar(&options.Unsets, "unset", options.Unsets, "Directly unset values in the spec")
		cmd.RegisterFlagCompletionFunc("unset", func(cmd *cobra.Command, args []string, toComplete string) ([]string, cobra.ShellCompDirective) {
			return nil, cobra.ShellCompDirectiveNoFileComp
		})
	}


-> featureflag.SpecOverrideFlag.Enabled()


File :             ./kops/cmd/kops/create_instancegroup.go 



if r == kopsapi.InstanceGroupRoleAPIServer && !featureflag.APIServerNodes.Enabled() {
			continue
		}


-> r == kopsapi.InstanceGroupRoleAPIServer && !featureflag.APIServerNodes.Enabled()


F

In [11]:
count_loc

{'APIServerNodes': 115,
 'AWSIPv6': 3,
 'AlphaAllowALI': 6,
 'AlphaAllowGCE': 27,
 'CacheNodeidentityInfo': 3,
 'ClusterAddons': 93,
 'EnableSeparateConfigBase': 7,
 'ExperimentalClusterDNS': 106,
 'GoogleCloudBucketACL': 56,
 'KeepLaunchConfigurations': 3,
 'KopsControllerStateStore': 0,
 'SkipEtcdVersionCheck': 15,
 'SpecOverrideFlag': 30,
 'Spotinst': 396,
 'SpotinstController': 15,
 'SpotinstHybrid': 103,
 'SpotinstOcean': 13,
 'TerraformJSON': 13,
 'TerraformManagedFiles': 23,
 'UseAddonOperators': 71,
 'VFSVaultSupport': 22,
 'VPCSkipEnableDNSSupport': 29}

In [12]:
print("\\begin{table}")
print("\\begin{tabular}{|c|c|c|c|}")
print("\\hline")
print("FT & \\#Files & \\#LOC & \\#Occurences \\\\ \\hline")
for kw in sorted(keywords):
    print(kw, "& ", str(count_file[kw]), "& ", str(count_loc[kw]), "& ", str(occurences[kw]), "\\\\ \\hline")
print("\\end{tabular}")
print("\\end{table}")

\begin{table}
\begin{tabular}{|c|c|c|c|}
\hline
FT & \#Files & \#LOC & \#Occurences \\ \hline
APIServerNodes &  8 &  115 &  16 \\ \hline
AWSIPv6 &  3 &  3 &  8 \\ \hline
AlphaAllowALI &  3 &  6 &  6 \\ \hline
AlphaAllowGCE &  7 &  27 &  10 \\ \hline
CacheNodeidentityInfo &  5 &  3 &  10 \\ \hline
ClusterAddons &  3 &  93 &  10 \\ \hline
EnableSeparateConfigBase &  2 &  7 &  4 \\ \hline
ExperimentalClusterDNS &  2 &  106 &  7 \\ \hline
GoogleCloudBucketACL &  2 &  56 &  3 \\ \hline
KeepLaunchConfigurations &  2 &  3 &  4 \\ \hline
KopsControllerStateStore &  2 &  0 &  4 \\ \hline
SkipEtcdVersionCheck &  2 &  15 &  6 \\ \hline
SpecOverrideFlag &  6 &  30 &  9 \\ \hline
Spotinst &  41 &  396 &  162 \\ \hline
SpotinstController &  2 &  15 &  4 \\ \hline
SpotinstHybrid &  5 &  103 &  12 \\ \hline
SpotinstOcean &  2 &  13 &  5 \\ \hline
TerraformJSON &  3 &  13 &  6 \\ \hline
TerraformManagedFiles &  4 &  23 &  7 \\ \hline
UseAddonOperators &  3 &  71 &  6 \\ \hline
VFSVaultSupport &  2 &  2